In [8]:
import requests
import json
from openai import AzureOpenAI
import logging

# Thêm logging configuration ở đầu file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Thiết lập API key cho Azure OpenAI và Airtable
azure_api_key = ""  # Thay bằng API key của bạn
airtable_api_key = ""  # Thay bằng Airtable API key
base_id = "app7mcJMf9O8GQRal"  # Base ID của bạn
table_name = "youtube_post"  # Table ID của bạn
view = "0_extract"  # View ID của bạn

azure_endpoint = "https://stepup.openai.azure.com"  # Thay bằng endpoint của bạn
api_version = "2024-08-01-preview"  # Phiên bản API
openai_model = "gpt-4o"  # Model OpenAI

# Thiết lập client Azure OpenAI
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=api_version
)

# Prompt hệ thống
system_prompt = """
Bạn là chuyên gia marketing, thành thạo phân tích video và social media \
Nhiệm vụ của bạn là khai thác insights từ nội dung các video và comment về đề tài học tiếng Anh\
Đọc và tìm ra những thông tin sau, trả kết quả dạng JSON, bằng tiếng Việt, nếu có trích dẫn thì để nguyên ngôn ngữ gốc\n\n

1. AI_filter: Kiểm tra xem nội dung này có đề cập tới việc học tiếng Anh hay không.\n
- english_learning: liên quan đến việc học tiếng Anh, hoặc giao tiếp tiếng Anh\n
- learning: việc học nói chung, không phải học tiếng Anh\n
- other: khác, không liên quan đến việc học\n\n

2. AI_purpose: Xác định mục đích của video này\n
- Promotional: video quảng cáo sản phẩm, nhằm nhắm đến chuyển đổi như giới thiệu sản phẩm, vào website...\n
- Mục đích khác nếu có \n\n

3. AI_product: \n
- Product: Xác định các sản phẩm/dịch vụ được nhắc tới trong video này nhằm mục đích quảng bá tăng chuyển đổi
- Solution: Tính năng, giải pháp, phương pháp học tiếng Anh được nhắc đến

4. AI_reverse: Phân tích nội dung video theo các chiều thông tin sau, trả về kết quả dạng bullet:\n
- Hook: câu hook mà người nói dùng để thu hút người nghe \n
- Message: Thông điệp chính mà người nói muốn truyền tải\n
- Audience: đối tượng mà người nói đang hướng tới hoặc thu hút chuyển đổi\n
- Insights: insight/vấn đề của người nghe mà người nói muốn chạm vào và giải quyết \n
- Outline: cấu trúc của video theo thứ tự thời gian này\n
- Technique: các kỹ thuật marketing mà người nói sử dụng trong việc làm content hoặc thiết kế video\n
- Research: các nghiên cứu khoa học được đề cập đến\n

5. AI_analysis: Đánh giá về video này dưới góc độ chuyên gia marketing\n

6. AI_other_insights: Ngoài những điều trên, bạn còn phát hiện ra điểm gì thú vị, độc đáo trong video này không?\n

Lưu ý: trả kết quả bằng tiếng Việt, nếu có trích dẫn thì để nguyên ngôn ngữ gốc


"""

def fetch_airtable_data(base_id, table_name, view, api_key, offset=None):
    try:
        url = f"https://api.airtable.com/v0/{base_id}/{table_name}"
        headers = {
            "Authorization": f"Bearer {api_key}",
        }
        params = {
            "view": view
        }
        if offset:
            params["offset"] = offset
            
        logger.info(f"Fetching data from Airtable with params: {params}")
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            logger.info(f"Successfully fetched {len(data.get('records', []))} records")
            return data
        else:
            logger.error(f"Failed to fetch data. Status code: {response.status_code}")
            logger.error(f"Response: {response.text}")
            return None
    except Exception as e:
        logger.error(f"Error fetching data: {str(e)}")
        return None

def update_airtable_record(base_id, table_name, api_key, record_id, fields):
    url = f"https://api.airtable.com/v0/{base_id}/{table_name}/{record_id}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {"fields": fields}
    response = requests.patch(url, headers=headers, json=data)
    if response.status_code != 200:
        logger.error(f"Failed to update record {record_id}. Status code: {response.status_code}, response: {response.text}")
    else:
        logger.info(f"Successfully updated record {record_id}")

def get_completion(prompt, record_id):
    try:
        logger.info(f"Sending request to OpenAI for record {record_id}")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            top_p=1,
            presence_penalty=0,
            frequency_penalty=0,
            max_tokens=4000
        )
        logger.info(f"Received response from OpenAI for record {record_id}")
        return response
    except Exception as e:
        logger.error(f"Error in OpenAI API call for record {record_id}: {e}")
        return None

def test_openai_connection():
    try:
        logger.info("Kiểm tra kết nối Azure OpenAI")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[{"role": "user", "content": "Hello"}],
            max_tokens=10
        )
        logger.info("Kết nối Azure OpenAI thành công")
        return True
    except Exception as e:
        logger.error(f"Lỗi kết nối Azure OpenAI: {str(e)}")
        return False

def format_reverse_to_bullets(data_dict):
    if not isinstance(data_dict, dict):
        return data_dict
        
    formatted_text = ""
    for key, value in data_dict.items():
        if isinstance(value, list):
            formatted_text += f"- {key}:\n"
            for item in value:
                formatted_text += f"  + {item}\n"
        else:
            formatted_text += f"- {key}: {value}\n"
    return formatted_text.strip()

def main():
    try:
        logger.info("Bắt đầu chạy chương trình")
        offset = None
        while True:
            logger.info("Đang fetch dữ liệu từ Airtable...")
            data = fetch_airtable_data(base_id, table_name, view, airtable_api_key, offset)

            if not data:
                logger.warning("Không lấy được dữ liệu từ Airtable")
                break

            records = data.get("records", [])
            logger.info(f"Số lượng records lấy được: {len(records)}")

            for record in records:
                record_id = record.get("id")
                logger.info(f"Đang xử lý record ID: {record_id}")
                
                fields = record.get("fields", {})
                title = fields.get("title", "")
                content_text = fields.get("description", "")
                transcript = fields.get("script", "")

                logger.info(f"Dữ liệu của record {record_id}:")
                logger.info(f"Title: {title[:50]}...")
                logger.info(f"Content: {content_text[:50]}...")
                logger.info(f"Script: {transcript[:50]}...")

                if content_text:  # Chỉ kiểm tra content_text
                    logger.info(f"Đang tạo prompt cho record {record_id}")
                    full_content = f"""
                    Tiêu đề video: {title}\n
                    Mô tả video từ người đăng: {content_text}\n
                    Transcript của video lấy từ audio: {transcript if transcript else 'Không có'}
                    """
                    
                    logger.info(f"Đang gọi OpenAI API cho record {record_id}")
                    response = get_completion(full_content, record_id)
                    
                    if response is None:
                        logger.warning(f"Không nhận được response từ OpenAI cho record {record_id}")
                        continue

                    try:
                        logger.info(f"Đang xử lý response cho record {record_id}")
                        ai_message_content = response.choices[0].message.content.strip()
                        logger.info(f"Response content: {ai_message_content[:100]}...")

                        # Loại bỏ markdown code block markers
                        if ai_message_content.startswith('```json'):
                            ai_message_content = ai_message_content[7:]
                        if ai_message_content.endswith('```'):
                            ai_message_content = ai_message_content[:-3]
                        
                        # Trim whitespace
                        ai_message_content = ai_message_content.strip()
                        
                        logger.info(f"Cleaned JSON content: {ai_message_content[:100]}...")
                        
                        # Parse JSON
                        logger.info("Đang parse JSON response")
                        parsed_json = json.loads(ai_message_content)
                        
                        # Chuẩn bị dữ liệu để update
                        logger.info("Đang chuẩn bị dữ liệu để update Airtable")
             
                        extracted_fields = {
                            "AI_filter": parsed_json.get("AI_filter", "no information"),
                            "AI_purpose": ", ".join(parsed_json.get("AI_purpose", [])) if isinstance(parsed_json.get("AI_purpose"), list) else parsed_json.get("AI_purpose", "no information"),
                            "AI_product": format_reverse_to_bullets(parsed_json.get("AI_product", {})),
                            "AI_reverse": format_reverse_to_bullets(parsed_json.get("AI_reverse", {})),
                            "AI_analysis": parsed_json.get("AI_analysis", "no information"),
                            "AI_other_insights": parsed_json.get("AI_other_insights", "no information")
                        }
                        
                        # Log extracted fields trước khi update
                        logger.info(f"Extracted fields: {json.dumps(extracted_fields, ensure_ascii=False)}")
                        
                        # Update Airtable
                        logger.info(f"Đang update record {record_id} vào Airtable")
                        update_airtable_record(base_id, table_name, airtable_api_key, record_id, extracted_fields)
                        logger.info(f"Đã update thành công record {record_id}")
                        
                    except json.JSONDecodeError as e:
                        logger.error(f"Lỗi parse JSON cho record {record_id}: {e}")
                        logger.error(f"JSON content causing error: {ai_message_content}")
                    except Exception as e:
                        logger.error(f"Lỗi xử lý response cho record {record_id}: {e}")
                else:
                    logger.warning(f"Thiếu content_text cho record {record_id}")

            offset = data.get("offset")
            if not offset:
                logger.info("Đã xử lý hết tất cả records")
                break

    except Exception as e:
        logger.error(f"Lỗi trong quá trình chạy: {str(e)}")
        raise e

if __name__ == "__main__":
    if test_openai_connection():
        main()
    else:
        logger.error("Không thể kết nối với Azure OpenAI, dừng chương trình")



2024-10-30 10:58:41,265 - INFO - Kiểm tra kết nối Azure OpenAI
2024-10-30 10:58:45,851 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 10:58:45,882 - INFO - Kết nối Azure OpenAI thành công
2024-10-30 10:58:45,911 - INFO - Bắt đầu chạy chương trình
2024-10-30 10:58:45,921 - INFO - Đang fetch dữ liệu từ Airtable...
2024-10-30 10:58:45,926 - INFO - Fetching data from Airtable with params: {'view': '0_extract'}
2024-10-30 10:58:47,419 - INFO - Successfully fetched 0 records
2024-10-30 10:58:47,420 - INFO - Số lượng records lấy được: 0
2024-10-30 10:58:47,421 - INFO - Đã xử lý hết tất cả records
